In [20]:
%run Format.ipynb
import ROOT as root
from array import array
from math import sin
from ROOT import gRandom
from ROOT import TFile
from ROOT import TPad, TLegend
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

In [21]:
isigma = 2
N_centr =5
N_hists = 5
centralities = ["0-88%","0-20%","20-40%","40-60%","60-88%"]
centrality = [0,9,0,1,2,3,4,5,6,9]
colors=[1,2,4,6,root.kGreen+2]
file_path="input/"
file_names="HeAu_19053_Artem_newcuts_fixed_pt.root"#"HeAu_19018_Artem_newcuts_E04.root"#HeAu_18968_Artem_newcuts
histo_names = ["eta_spectrum_pbsc_hist_FG11","eta_spectrum_pbsc_hist_BG11","r_2_eta_spectrum_pbsc_hist_FG11","r_2_eta_spectrum_pbsc_hist_BG11",
"r_3_eta_spectrum_pbsc_hist_FG11","r_3_eta_spectrum_pbsc_hist_BG11","r_4_eta_spectrum_pbsc_hist_FG11","r_4_eta_spectrum_pbsc_hist_BG11",
"emcid_eta_spectrum_pbsc_hist_FG11","emcid_eta_spectrum_pbsc_hist_BG11"]

In [22]:
hists_eta_FG_read, hists_eta_BG_read = [], []
hist_stat_read=[]
infile = root.TFile.Open(file_path+file_names, "read")

for ihist in range(N_hists):
    hists_eta_FG_read_centr, hists_eta_BG_read_centr = [], []
    hist_stat_centr = []
    for icent in range(N_centr):
        print(histo_names[ihist*2])
        hist_FG = infile.Get("c00_z00_r00/"+histo_names[ihist*2])
        hist_FG.Add(hist_FG,-1)
        hist_FG.SetDirectory(root.nullptr)
        hist_BG = infile.Get("c00_z00_r00/"+histo_names[ihist*2+1])
        hist_BG.Add(hist_BG,-1)
        hist_BG.SetDirectory(root.nullptr)
        hist_stat = infile.Get("c00_z00_r00/PoolStatistics")
        hist_stat.Add(hist_stat,-1)
        hist_stat.SetDirectory(root.nullptr)
        for i in range(centrality[icent*2],centrality[icent*2+1]+1):
            for j in range(6):
                hist_FG.Add(infile.Get(f"c0{i}_z0{j}_r00/"+histo_names[ihist*2]))
                hist_BG.Add(infile.Get(f"c0{i}_z0{j}_r00/"+histo_names[ihist*2+1]))
                hist_stat.Add(infile.Get(f"c0{i}_z0{j}_r00/PoolStatistics"))
        hists_eta_FG_read_centr.append(hist_FG)
        hists_eta_BG_read_centr.append(hist_BG)
        hist_stat_centr.append(hist_stat)
    hists_eta_FG_read.append(hists_eta_FG_read_centr)
    hists_eta_BG_read.append(hists_eta_BG_read_centr)
    hist_stat_read.append(hist_stat_centr)


infile.Close()

eta_spectrum_pbsc_hist_FG11


eta_spectrum_pbsc_hist_FG11
eta_spectrum_pbsc_hist_FG11
eta_spectrum_pbsc_hist_FG11
eta_spectrum_pbsc_hist_FG11
r_2_eta_spectrum_pbsc_hist_FG11
r_2_eta_spectrum_pbsc_hist_FG11
r_2_eta_spectrum_pbsc_hist_FG11
r_2_eta_spectrum_pbsc_hist_FG11
r_2_eta_spectrum_pbsc_hist_FG11
r_3_eta_spectrum_pbsc_hist_FG11
r_3_eta_spectrum_pbsc_hist_FG11
r_3_eta_spectrum_pbsc_hist_FG11
r_3_eta_spectrum_pbsc_hist_FG11
r_3_eta_spectrum_pbsc_hist_FG11
r_4_eta_spectrum_pbsc_hist_FG11
r_4_eta_spectrum_pbsc_hist_FG11
r_4_eta_spectrum_pbsc_hist_FG11
r_4_eta_spectrum_pbsc_hist_FG11
r_4_eta_spectrum_pbsc_hist_FG11
emcid_eta_spectrum_pbsc_hist_FG11
emcid_eta_spectrum_pbsc_hist_FG11
emcid_eta_spectrum_pbsc_hist_FG11
emcid_eta_spectrum_pbsc_hist_FG11
emcid_eta_spectrum_pbsc_hist_FG11


In [23]:

pt = array( 'd' )
pt_boarders = array( 'd', [1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,6.0,8.0,10.,20.])
pt_err = array( 'd' )
N_pt = len(pt_boarders) - 1
for i in range(N_pt):
    pt.append( (pt_boarders[i]+pt_boarders[i+1])/2)
    pt_err.append(0)
pt_boarders.append(20.)
print("pt = ", pt,"\npt_boarders = ",pt_boarders)

pt =  array('d', [1.25, 1.75, 2.25, 2.75, 3.25, 3.75, 4.25, 4.75, 5.5, 7.0, 9.0, 15.0]) 
pt_boarders =  array('d', [1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 6.0, 8.0, 10.0, 20.0, 20.0])


In [24]:
keffs= [0.99,0.7,0.7,0.8,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,]

def emc_fitter(InHistFG, InHistBG, centr, seed=1, sigma=2, fits_mean=0, fits_sigma=0):
    N_gaus=1
    seed0 = seed
    seed=seed*N_pt

    hist_pt_bin = InHistFG[icent].ProjectionY("kek")

    Mean_local =  array( 'd' )
    Sigma_local =  array( 'd' )
    Mean_local_err =  array( 'd' )
    Sigma_local_err =  array( 'd' )

    int_fg_local =  array( 'd' )
    int_cbg_local =  array( 'd' )
    int_rbg_local =  array( 'd' )
 
    for p in range(N_pt):
        c1 = root.TCanvas(f"c1{seed+p}",f"c1{seed+p}",720,360)

        eta_FG_porj = InHistFG[centr].ProjectionX(f"FG{seed*N_pt+p}",hist_pt_bin.FindBin(pt_boarders[p]),hist_pt_bin.FindBin(pt_boarders[p+1])-1)
        eta_BG_porj = InHistBG[centr].ProjectionX(f"BG{seed*N_pt+p}",hist_pt_bin.FindBin(pt_boarders[p]),hist_pt_bin.FindBin(pt_boarders[p+1])-1)
        eta_CG_porj = InHistFG[centr].ProjectionX(f"CG{seed*N_pt+p}",hist_pt_bin.FindBin(pt_boarders[p]),hist_pt_bin.FindBin(pt_boarders[p+1])-1)

        eta_FG_porj.Rebin(10)
        eta_BG_porj.Rebin(10)
        eta_CG_porj.Rebin(10)

        left = eta_FG_porj.FindBin(0.7)
        right = eta_FG_porj.FindBin(0.8)

        eta_BG_porj.Scale(eta_FG_porj.Integral(left,right)/(eta_BG_porj.Integral(left,right)+0.1))
        eta_FG_porj.Add(eta_BG_porj,-keffs[p]*0.995)
        eta_FG_porj.SetAxisRange(0.35,0.8)
        eta_FG_porj.SetLineColor(1)


        par_tot = array( 'd', (3*N_gaus+3)*[2.] )
        g_gaus= root.TF1( f'g_gaus{seed+p}', 'gaus',  0.5,0.61)
        g_pol = root.TF1( f'g_pol{seed+p}', '[0]+[1]*x+[2]*x*x',  0.35,0.8)
        Format_Func(g_gaus,3,4)
        Format_Func(g_pol,3,2)

        total = root.TF1( f'total_pos{seed+p}', 'gaus(0)+[3]+[4]*x+[5]*x*x',0.4,0.8)
        Format_Func(total,2,1)


        if fits_mean!=0 and fits_sigma!=0:
            mean_ii=fits_sigma.Eval(pt[p])
            sigma_ii=fits_sigma.Eval(pt[p])
            g_gaus.SetParameter(1,sigma_ii)
            g_gaus.SetParameter(2,sigma_ii)
            g_gaus.SetParLimits(1,mean_ii*0.8-0.05,mean_ii*1.2+0.05)
            g_gaus.SetParLimits(2,sigma_ii*0.5,sigma_ii*1.5)
        else:
            g_gaus.SetParameter(1,0.55)
            g_gaus.SetParameter(2,0.03)
            g_gaus.SetParLimits(1,0.505,0.6)
            g_gaus.SetParLimits(2,0.025,0.05)


        #emc_var_porj.GetXaxis().SetRangeUser(-1.4, 0.5);
        eta_FG_porj.Draw()
        eta_FG_porj.Fit( g_gaus, 'QR')
        eta_FG_porj.Fit( g_pol, 'QR+' )

        par_gaus = g_gaus.GetParameters()
        par_pol = g_pol.GetParameters()

        par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5] = par_gaus[0], par_gaus[1], par_gaus[2], par_pol[0], par_pol[1], par_pol[2]

        total.SetParameters( par_tot )

        total.SetParLimits(1,0.505,0.6)
        total.SetParLimits(2,0.025,0.05)
        
        Format_Graph(eta_FG_porj,20,1,1,1,1,1,1)
        eta_FG_porj.Draw("P")
        eta_FG_porj.Fit( total, 'QR' )

        par_tot = total.GetParameters()

        par_gaus[0], par_gaus[1], par_gaus[2], par_pol[0], par_pol[1], par_pol[2] = par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5]

        g_gaus.SetParameters(par_gaus)
        g_pol.SetParameters(par_pol)

        g_gaus.Draw('same')
        g_pol.Draw('same')

        Mean_local.append(total.GetParameter(1))
        Sigma_local.append(total.GetParameter(2))
        Mean_local_err.append(total.GetParError(1))
        Sigma_local_err.append(total.GetParError(2))

        #mean_int = total.GetParameter(1)
        #sigma_int = abs(total.GetParameter(2))

        left_int = eta_FG_porj.FindBin(0.48)
        right_int = eta_FG_porj.FindBin(0.62)+1
        
        FG, CBG, RBG = 0.1, 0.1, 0.1
        for i in range(left_int,right_int):
            FG += eta_FG_porj.GetBinContent(i) - g_pol.Eval(eta_FG_porj.GetBinCenter(i))
            CBG += eta_CG_porj.GetBinContent(i)
            RBG += g_pol.Eval(eta_FG_porj.GetBinCenter(i))


        int_fg_local.append(FG)
        int_cbg_local.append(CBG)
        int_rbg_local.append(RBG)
            
        #m2_pos.Draw("same") m2_neg.Draw("same")
        m2_name="fits/"+str(seed0)+"/"+str(centr)+"/eta_"+str(round(pt[p],1))

        if fits_mean==0:
            c1.SaveAs( "output/" + m2_name + ".png" )
        else:
            c1.SaveAs( "output/" + m2_name + "upd.png" )

    return [Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, int_fg_local, int_cbg_local, int_rbg_local]


In [25]:
def emc_centr_fitter(InHistFG, InHistBG, centralities, seed=1, sigma=2, fits_mean=0, fits_sigma=0):
    hist_ser = []
    for centr in centralities:
        hist_ser.append(emc_fitter(InHistFG, InHistBG, centr, seed, sigma, fits_mean, fits_sigma))
    return hist_ser

In [26]:
import multiprocess as mp

Ntr = N_hists

pool = mp.Pool(Ntr)
output_array = pool.starmap(emc_centr_fitter, [(hists_eta_FG_read[i], hists_eta_BG_read[i], [0,1,2,3,4], i, isigma) for i in range(Ntr)])
pool.close()

In [27]:
Means, Mean_errs, Sigmas, Sigma_errs, Int_FG, Int_CBG, Int_RBG = [], [], [], [], [], [], []
for i in range(Ntr):
    Means0, Mean_errs0, Sigmas0, Sigma_errs0, Int_FG0, Int_CBG0, Int_RBG0 = [], [], [], [], [], [], []
    for j in range(N_centr):
        Means0.append(output_array[i][j][0])
        Mean_errs0.append(output_array[i][j][1])
        Sigmas0.append(output_array[i][j][2])
        Sigma_errs0.append(output_array[i][j][3])
        Int_FG0.append(output_array[i][j][4])
        Int_CBG0.append(output_array[i][j][5])
        Int_RBG0.append(output_array[i][j][6])
    Means.append(Means0)
    Mean_errs.append(Mean_errs0)
    Sigmas.append(Sigmas0)
    Sigma_errs.append(Sigma_errs0)
    Int_FG.append(Int_FG0)
    Int_CBG.append(Int_CBG0)
    Int_RBG.append(Int_RBG0)

In [28]:
h2=[]
c3 = root.TCanvas(f"c3",f"c3",500*3,500*2)
c3.Divide(3,2)
g_sigma = [[[]]*N_centr]*N_hists
g_sigma_inter = [[[]]*6]*N_hists
names = ["emc_r_sdphi","emc_min_r_sdphi","emc_cnt_sdphi","emc_r_sdz","emc_min_r_sdz","emc_cnt_sdz"]
legends =[]
for itype in range(N_hists):
    c3.cd(itype+1)
    h2.append(Format_Pad_old( 0.05, 16, 0.02, 0.05, "p_{T} (GeV/#it{c})", "Sigmas", 1.1, 1.1, 0.03, 0.03, "" ))
    h2[-1].Draw()
    legends.append(root.TLegend(0.3,0.5,0.7,0.9,names[itype]))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetLineWidth(0)
    for icentr in range(0,N_centr):
        
        g_sigma[itype][icentr].append(root.TGraphErrors(N_pt, pt, Sigmas[itype][icentr],pt_err,Sigma_errs[itype][icentr]))
               
        Format_Graph(g_sigma[itype][icentr][-1],20,1, colors[icentr], 3, colors[icentr],1,1)
        g_sigma[itype][icentr][-1].Draw('P')
      
        legends[-1].AddEntry(g_sigma[itype][icentr][-1],centralities[icentr],"p")
        
    legends[-1].Draw()
c3.Draw()
c3.SaveAs("output/eta_sigma.png")

In [29]:
h3=[]
c4 = root.TCanvas(f"c4",f"c4",500*3,500*2)
c4.Divide(3,2)
g_mean = [[[]]*N_centr]*N_hists
names = ["emc_r_sdphi","emc_min_r_sdphi","emc_cnt_sdphi","emc_r_sdz","emc_min_r_sdz","emc_cnt_sdz"]
legends =[]
for itype in range(N_hists):
    c4.cd(itype+1)
    h3.append(Format_Pad_old( 0., 16, 0.50, 0.6, "p_{T} (GeV/#it{c})", "Means", 1.1, 1.1, 0.03, 0.03, "" ))
    h3[-1].Draw()
    legends.append(root.TLegend(0.3,0.5,0.7,0.9,names[itype]))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetLineWidth(0)
    for icentr in range(0,N_centr):

        g_mean[itype][icentr].append(root.TGraphErrors(N_pt, pt, Means[itype][icentr],pt_err,Mean_errs[itype][icentr]))
        Format_Graph(g_mean[itype][icentr][-1],20,1, colors[icentr], 3, colors[icentr],1,1)
        
        g_mean[itype][icentr][-1].Draw('P')
        legends[-1].AddEntry(g_mean[itype][icentr][-1],centralities[icentr],"p")
        
    legends[-1].Draw()
c4.Draw()
c4.SaveAs("output/eta_mean.png")

In [39]:
FG_BG, FG_BG_err = [], []
for icase in range(24):
    FG_BG0, FG_BG_err0 = [], []
    for icentr in range(N_centr):
        FG_BG_loc =  array( 'd' )
        FG_BG_err_loc =  array( 'd' )
        for ipt in range(N_pt):
            
            if icase < 4:
                FG_BG_loc.append(Int_FG[icase+1][icentr][ipt]/abs(Int_FG[0][icentr][ipt]+0.1))
                FG_BG_err_loc.append(FG_BG_loc[-1]*abs(1./Int_FG[icase+1][icentr][ipt]+1./Int_FG[0][icentr][ipt])**0.5)
            if icase >= 4 and icase < 8:
                FG_BG_loc.append(Int_RBG[icase-3][icentr][ipt]/Int_RBG[0][icentr][ipt])
                FG_BG_err_loc.append(FG_BG_loc[-1]*(1./abs(Int_RBG[icase-3][icentr][ipt])+1./abs(Int_RBG[0][icentr][ipt]))**0.5)
            if icase >= 8 and icase < 12:
                FG_BG_loc.append((Int_CBG[icase-7][icentr][ipt]-Int_FG[icase-7][icentr][ipt])/(Int_CBG[0][icentr][ipt]-Int_FG[0][icentr][ipt]))
                FG_BG_err_loc.append(FG_BG_loc[-1]*abs(1./Int_CBG[icase-7][icentr][ipt]+1./Int_CBG[0][icentr][ipt])**0.5)
            if icase >= 12 and icase < 16:
                FG_BG_loc.append(Int_FG[icase-11][icentr][ipt]/Int_FG[0][icentr][ipt]/((Int_RBG[icase-11][icentr][ipt])/(Int_RBG[0][icentr][ipt])))
                FG_BG_err_loc.append(FG_BG_loc[-1]*abs(1./(Int_FG[icase-11][icentr][ipt]+0.1)+1./abs(Int_RBG[icase-11][icentr][ipt]+0.1)+1./(Int_FG[0][icentr][ipt]+0.1)+1./abs(Int_RBG[0][icentr][ipt]))**0.5)
            if icase >= 16 and icase < 20:
                FG_BG_loc.append(Int_FG[icase-15][icentr][ipt]/Int_FG[0][icentr][ipt]/((Int_CBG[icase-15][icentr][ipt])/(Int_CBG[0][icentr][ipt])))
                FG_BG_err_loc.append(FG_BG_loc[-1]*abs(1./(Int_FG[icase-15][icentr][ipt]+0.1)+1./(Int_CBG[icase-15][icentr][ipt]+0.1)+1./(Int_FG[0][icentr][ipt]+0.1)+1./(Int_CBG[0][icentr][ipt]+0.1))**0.5)
            if icase >= 20 and icase < 24:
                FG_BG_loc.append((Int_CBG[icase-19][icentr][ipt])**(0.5)/Int_FG[icase-19][icentr][ipt]/(Int_CBG[0][icentr][ipt])**(0.5)*Int_FG[0][icentr][ipt])
                FG_BG_err_loc.append(FG_BG_loc[-1]*abs(1./Int_FG[icase-19][icentr][ipt]+1./Int_CBG[icase-19][icentr][ipt]+1./Int_FG[0][icentr][ipt]+1./Int_CBG[0][icentr][ipt])**0.5)
            
        FG_BG0.append(FG_BG_loc)
        FG_BG_err0.append(FG_BG_err_loc)
    FG_BG.append(FG_BG0)
    FG_BG_err.append(FG_BG_err0)

In [40]:
h5=[]
c5 = root.TCanvas(f"c5",f"c5",500*2,500*2)
c5.Divide(2,2)
g_mean = [[[]]*12]*12
names = ["#eta sigma cut", "#eta r_min<3", "#eta r_cnt", "#eta emcid"]
min_max = [0.5,1.2, 0.5,1.2,0.5,1.2,0.5,1.2]
legends =[]
for itype in range(4):
    c5.cd(itype+1)
    h5.append(Format_Pad_old( 0.25, 10, min_max[2*itype], min_max[2*itype+1], "p_{T} (GeV/#it{c})", "Raw Yield cut/default", 1.1, 1.1, 0.03, 0.03, "" ))
    h5[-1].Draw()
    legends.append(root.TLegend(0.4,0.6,0.9,0.9))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetTextAlign(23)
    legends[-1].SetLineWidth(0)
    legends[-1].SetTextSize(0.05)
    for icentr in range(0,N_centr):

        g_mean[itype][icentr].append(root.TGraphErrors(N_pt, pt, FG_BG[itype][icentr],pt_err,FG_BG_err[itype][icentr]))
        Format_Graph(g_mean[itype][icentr][-1], 20, 1, colors[icentr], 3, colors[icentr], 1, 1)

        g_mean[itype][icentr][-1].Draw('P')
        g_mean[itype][icentr][-1].Fit("pol0","Q")
        f = g_mean[itype][icentr][-1].GetFunction("pol0")
        f.SetLineColor(colors[icentr])
        legends[-1].AddEntry(g_mean[itype][icentr][-1],names[itype]+" "+centralities[icentr],"p")
        
    legends[-1].Draw()
c5.Draw()
c5.SaveAs("output/eta_Yield_ratio_cut_default.png")

In [41]:
h6=[]
c6 = root.TCanvas(f"c6",f"c6",500*2,500*2)
c6.Divide(2,2)
g_mean = [[[]]*12]*12
names = ["#eta sigma cut", "#eta r_min<3", "#eta r_cnt", "#eta emcid"]
legends =[]
for itype in range(4):
    c6.cd(itype+1)
    h6.append(Format_Pad_old( 0.25, 10, 0.0, 1.1, "p_{T} (GeV/#it{c})", "residial BG cut/defualt", 1.1, 1.1, 0.03, 0.03, "" ))
    h6[-1].Draw()
    legends.append(root.TLegend(0.4,0.6,0.9,0.9))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetTextAlign(23)
    legends[-1].SetLineWidth(0)
    legends[-1].SetTextSize(0.05)
    for icentr in range(0,N_centr):

        g_mean[itype][icentr].append(root.TGraphErrors(N_pt, pt, FG_BG[itype+4][icentr],pt_err,FG_BG_err[itype+4][icentr]))
        Format_Graph(g_mean[itype][icentr][-1], 20, 1, colors[icentr], 3, colors[icentr], 1, 1)

        g_mean[itype][icentr][-1].Draw('P')
        g_mean[itype][icentr][-1].Fit("pol0","Q")
        f = g_mean[itype][icentr][-1].GetFunction("pol0")
        f.SetLineColor(colors[icentr])
        legends[-1].AddEntry(g_mean[itype][icentr][-1],names[itype]+" "+centralities[icentr],"p")
        
    legends[-1].Draw()
c6.Draw()
c6.SaveAs("output/eta_residial_ratio_cut_default.png")

In [42]:
h6=[]
c6 = root.TCanvas(f"c6",f"c6",500*2,500*2)
c6.Divide(2,2)
g_mean = [[[]]*12]*12
names = ["#eta sigma cut", "#eta r_min<3", "#eta r_cnt", "#eta emcid"]
legends =[]
for itype in range(4):
    c6.cd(itype+1)
    h6.append(Format_Pad_old( 0.25, 10, 0.4, 1.1, "p_{T} (GeV/#it{c})", "total BG cut/defualt", 1.1, 1.1, 0.03, 0.03, "" ))
    h6[-1].Draw()
    legends.append(root.TLegend(0.4,0.6,0.9,0.9))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetTextAlign(23)
    legends[-1].SetLineWidth(0)
    legends[-1].SetTextSize(0.05)
    for icentr in range(0,N_centr):

        g_mean[itype][icentr].append(root.TGraphErrors(N_pt, pt, FG_BG[itype+8][icentr],pt_err,FG_BG_err[itype+8][icentr]))
        Format_Graph(g_mean[itype][icentr][-1], 20, 1, colors[icentr], 3, colors[icentr], 1, 1)

        g_mean[itype][icentr][-1].Draw('P')
        g_mean[itype][icentr][-1].Fit("pol0","Q")
        f = g_mean[itype][icentr][-1].GetFunction("pol0")
        f.SetLineColor(colors[icentr])
        legends[-1].AddEntry(g_mean[itype][icentr][-1],names[itype]+" "+centralities[icentr],"p")
        
    legends[-1].Draw()
c6.Draw()
c6.SaveAs("output/eta_BG_ratio_cut_default.png")

In [43]:
h6=[]
c6 = root.TCanvas(f"c6",f"c6",500*2,500*2)
c6.Divide(2,2)
g_mean = [[[]]*12]*12
names = ["#eta sigma cut", "#eta r_min<3", "#eta r_cnt", "#eta emcid"]
legends =[]
for itype in range(4):
    c6.cd(itype+1)
    h6.append(Format_Pad_old( 0.25, 10, 1.0, 2.6, "p_{T} (GeV/#it{c})", "FG/resid cut/default", 1.1, 1.1, 0.03, 0.03, "" ))
    h6[-1].Draw()
    legends.append(root.TLegend(0.4,0.6,0.9,0.9))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetTextAlign(23)
    legends[-1].SetLineWidth(0)
    legends[-1].SetTextSize(0.05)
    for icentr in range(0,N_centr):

        g_mean[itype][icentr].append(root.TGraphErrors(N_pt, pt, FG_BG[itype+12][icentr],pt_err,FG_BG_err[itype+12][icentr]))
        Format_Graph(g_mean[itype][icentr][-1], 20, 1, colors[icentr], 3, colors[icentr], 1, 1)

        g_mean[itype][icentr][-1].Draw('P')
        g_mean[itype][icentr][-1].Fit("pol0","Q")
        f = g_mean[itype][icentr][-1].GetFunction("pol0")
        f.SetLineColor(colors[icentr])
        legends[-1].AddEntry(g_mean[itype][icentr][-1],names[itype]+" "+centralities[icentr],"p")
        
    legends[-1].Draw()
c6.Draw()
c6.SaveAs("output/eta_FG_resid_ratio_cut_default.png")

In [44]:
h6=[]
c6 = root.TCanvas(f"c6",f"c6",500*2,500*2)
c6.Divide(2,2)
g_mean = [[[]]*12]*12
names =  ["#eta sigma cut", "#eta r_min<3", "#eta r_cnt", "#eta emcid"]
legends =[]
for itype in range(4):
    c6.cd(itype+1)
    h6.append(Format_Pad_old( 0.25, 10, .0, 5.6, "p_{T} (GeV/#it{c})", "FG/BG_tot cut/default", 1.1, 1.1, 0.03, 0.03, "" ))
    h6[-1].Draw()
    legends.append(root.TLegend(0.4,0.6,0.9,0.9))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetTextAlign(23)
    legends[-1].SetLineWidth(0)
    legends[-1].SetTextSize(0.05)
    for icentr in range(0,N_centr):

        g_mean[itype][icentr].append(root.TGraphErrors(N_pt, pt, FG_BG[itype+16][icentr],pt_err,FG_BG_err[itype+16][icentr]))
        Format_Graph(g_mean[itype][icentr][-1], 20, 1, colors[icentr], 3, colors[icentr], 1, 1)

        g_mean[itype][icentr][-1].Draw('P')
        g_mean[itype][icentr][-1].Fit("pol0","Q")
        f = g_mean[itype][icentr][-1].GetFunction("pol0")
        f.SetLineColor(colors[icentr])
        legends[-1].AddEntry(g_mean[itype][icentr][-1],names[itype]+" "+centralities[icentr],"p")
        
    legends[-1].Draw()
c6.Draw()
c6.SaveAs("output/eta_FG_BG_ratio_cut_default.png")

In [45]:
h6=[]
c6 = root.TCanvas(f"c6",f"c6",500*2,500*2)
c6.Divide(2,2)
g_mean = [[[]]*12]*12
min_max = [0.0,1.3, 0.0,1.3, 0.,1.3, 0.,1.3]
names = ["#eta sigma cut", "#eta r_min<3", "#eta r_cnt", "#eta emcid"]
legends =[]
for itype in range(4):
    c6.cd(itype+1)
    h6.append(Format_Pad_old( 0.25, 10, min_max[itype*2], min_max[itype*2+1], "p_{T} (GeV/#it{c})", "#delta_{Yield }  cut/default", 1.1, 1.1, 0.03, 0.03, "" ))
    h6[-1].Draw()
    legends.append(root.TLegend(0.4,0.6,0.9,0.9))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetTextAlign(23)
    legends[-1].SetLineWidth(0)
    legends[-1].SetTextSize(0.05)
    for icentr in range(0,N_centr):

        g_mean[itype][icentr].append(root.TGraphErrors(N_pt, pt, FG_BG[itype+20][icentr],pt_err,FG_BG_err[itype+20][icentr]))
        Format_Graph(g_mean[itype][icentr][-1], 20, 1, colors[icentr], 3, colors[icentr], 1, 1)

        g_mean[itype][icentr][-1].Draw('P')
        g_mean[itype][icentr][-1].Fit("pol0","Q")
        f = g_mean[itype][icentr][-1].GetFunction("pol0")
        f.SetLineColor(colors[icentr])
        legends[-1].AddEntry(g_mean[itype][icentr][-1],names[itype]+" "+centralities[icentr],"p")
        
    legends[-1].Draw()
c6.Draw()
c6.SaveAs("output/eta_stat_error_ratio_cut_default.png")

In [ ]:
!jupyter nbconvert --to html emc_r_HeAu.ipynb --HTMLExporter.theme=dark;

[NbConvertApp] Converting notebook emc_r_HeAu.ipynb to html
/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)
[NbConvertApp] Writing 1363597 bytes to emc_r_HeAu.html
